Провозившись с попыткой использовать самую последнюю версию torchtext (а именно с 0.12.0), я понял, что это не выход: туториалов перехода к последней версии просто нет (под туториалом я подразумеваю что-то типо такого https://satyajitghana.github.io/2021/05/torchtext-new-api.html). Поэтому я возьму и максимально упрощу себе жизнь, поставив старые версии нужных мне библиотек.

In [ ]:
# Поставим ласт версию, где ещё было все то, что используется в этом ноутбуке
!pip install torchtext==0.8.1
# В spacy нет русского
!python -m spacy download en

     |████████████████████████████████| 7.0 MB 4.5 MB/s 
     |████████████████████████████████| 776.8 MB 17 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.12.0
    Uninstalling torchtext-0.12.0:
      Successfully uninstalled torchtext-0.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.7.1 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.7.1 which is incompatible.
     |████████████████████████████████| 12.0 MB 5.3 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/us

После обязательно перезапустить среду выполнения!

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import numpy as np

from tqdm.autonotebook import tqdm

# Сентимент анализ

Домашка — классифицировать отзывы с IMDB на положительный / отрицательный только по тексту.

<img src="https://github.com/bentrevett/pytorch-sentiment-analysis/raw/bf8cc46e4823ebf9af721b595501ad6231c73632/assets/sentiment1.png">

Суть такая же, только нужно предобработать тексты — каждому слову сопоставить обучаемый вектор (embedding), который пойдёт дальше в RNN.

In [ ]:
from torchtext import data


SEED = 1234

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField(dtype=torch.float)

/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: LabelField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Выше вылезли всякие warnings, в которых говорится, что скоро всё используемое в этом блокноте перейдет torchtext.legacy, не будем на это обращать внимание!

In [ ]:
from torchtext import datasets


train_data, test_data = datasets.IMDB.splits(TEXT, LABEL, root="./data")

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:03<00:00, 21.8MB/s]
/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


In [ ]:
ls -lh data/imdb/aclImdb/

total 1.7M
-rw-r--r-- 1 7297 1000 882K Jun 11  2011 imdbEr.txt
-rw-r--r-- 1 7297 1000 827K Apr 12  2011 imdb.vocab
-rw-r--r-- 1 7297 1000 4.0K Jun 26  2011 README
drwxr-xr-x 4 7297 1000 4.0K Apr 12  2011 test/
drwxr-xr-x 5 7297 1000 4.0K Jun 26  2011 train/


In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [ ]:
# То есть элементом является один отзыв
print(vars(train_data.examples[0]))

{'text': ['THE', 'VAN', 'is', 'a', 'simple', 'teensploitation', 'picture', 'made', 'especially', 'for', 'the', 'drive', 'in', 'that', 'goes', 'out', 'of', 'it', "'s", 'way', 'to', 'make', 'you', 'feel', 'comfortable', ',', 'providing', 'many', 'opportunities', 'to', 'laugh', 'and', 'cry', 'with', 'your', 'friends', '.', 'Danny', 'Devito', 'has', 'a', 'small', 'yet', 'plentiful', 'role', 'as', 'the', 'manager', 'of', 'a', 'car', 'wash', 'and', 'almost', 'steals', 'the', 'show', '!', 'All', 'the', 'leads', 'are', 'well', 'acted', ',', 'the', 'characters', 'complex', 'and', 'the', 'directing', 'quite', 'competent', 'for', 'this', 'type', 'of', 'picture', '.', 'A', 'Crown', 'International', 'Release', '.'], 'label': 'pos'}


In [ ]:
# Посмотри на длины разных отзывов: они, скорее всего, будут разные.
print(len(vars(train_data.examples[0])['text']), len(vars(train_data.examples[1])['text']))

84 135


In [ ]:
train_data.examples[0]

In [ ]:
# Сделаем еще eval
import random

# default split_ratio=0.7
train_data, valid_data = train_data.split(random_state=random.seed(SEED))

In [ ]:
# Сделаем словарь: будем использовать в нем 25000 самых встречающихся слов
TEXT.build_vocab(train_data, max_size=25000)
LABEL.build_vocab(train_data)

In [ ]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [ ]:
vars(LABEL.vocab)

{'freqs': Counter({'neg': 8810, 'pos': 8690}),
 'itos': ['neg', 'pos'],
 'stoi': defaultdict(None, {'neg': 0, 'pos': 1}),
 'unk_index': None,
 'vectors': None}

Почему 25002, а не 25000?
Потому что $<unk>$ и $<pad>$.

Так как в словар мы добавли только 25000 самых встречающихся слов, то остальные мы просто заменили на $<unk>$.

$<pad>$ же нужен для дополнения предложений в batch до одной длины.

<img src="https://github.com/bentrevett/pytorch-sentiment-analysis/raw/bf8cc46e4823ebf9af721b595501ad6231c73632/assets/sentiment6.png" width="160">



In [ ]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 202627), (',', 191383), ('.', 165682), ('a', 109305), ('and', 108923), ('of', 101047), ('to', 93345), ('is', 75851), ('in', 61216), ('I', 54394), ('it', 53362), ('that', 48990), ('"', 44734), ("'s", 43070), ('this', 42150), ('-', 37086), ('/><br', 35418), ('was', 35143), ('as', 30324), ('with', 29944)]


* stoi (string to int)
* itos (int to string)

In [ ]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is']


In [ ]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


In [ ]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# собираем батчи так, чтобы в каждом батче были примеры наиболее похожей длины
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE,
    device=device)

/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


## Делаем модель

<img src="https://github.com/bentrevett/pytorch-sentiment-analysis/raw/bf8cc46e4823ebf9af721b595501ad6231c73632/assets/sentiment7.png" width="450">

* В эмбеддер (emb = [torch.nn.Embedding(num_embeddings, embedding_dim)](https://pytorch.org/docs/stable/nn.html?highlight=embedding#torch.nn.Embedding)) запихиваем тензор размерностью **[sentence length, batch size]**
* Эмбеддер возвращает тензор размерностью **[sentence length, batch size, embedding dim]**
* RNN (torch.nn.RNN(embedding_dim, hidden_dim)) возвращает 2 тензора, *output* размера [sentence length, batch size, hidden dim] и *hidden* размера [1, batch size, hidden dim]

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    # Очень странно, что здесь не было forward. Раз не было, добавим.
    def forward(self, text):

        #text.shape = [sent len, batch size]
        embedded = self.embedding(text)

        #embedded.shape = [sent len, batch size, emb dim]
        output, hidden = self.rnn(embedded)
    
        #output.shape = [sent len, batch size, hid dim]
        #hidden.shape = [1, batch size, hid dim]
    
        #         assert torch.equal(output[-1,:,:], hidden.squeeze(0))
    
        return self.fc(hidden.squeeze(0))

In [ ]:
INPUT_DIM = len(TEXT.vocab)
OUTPUT_DIM = 2 # два класса - два выхода

# Так как стартовых значений для нижезаданных параметров нет, то пусть будет так
EMBEDDING_DIM = 128
HIDDEN_DIM = 256
N_EPHOCS = 25

In [ ]:
model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def get_accuracy(logit, target, batch_size):
    corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
    accuracy = 100.0 * corrects/batch_size
    return accuracy.item()


def evaluate(model, valid_iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    
    with torch.no_grad():
        for data in valid_iterator:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.type(torch.LongTensor).to(device)

            outputs = model(inputs)

            loss = criterion(outputs, labels)
            epoch_loss += loss.detach().item()
            epoch_acc += get_accuracy(outputs, labels, BATCH_SIZE)

    return epoch_loss/len(valid_iterator), epoch_acc/len(valid_iterator)

In [ ]:
for epoch in tqdm(range(N_EPHOCS)):
    train_running_loss = 0.0
    train_running_acc = 0.0
    model.train()
    
    for data in tqdm(train_iterator, leave=False, total=len(train_iterator)):
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.type(torch.LongTensor).to(device)

        # forward + backward + optimize
        outputs = model(inputs)
        
        # loss = criterion(outputs, labels)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_running_loss += loss.detach().item()
        train_running_acc += get_accuracy(outputs, labels, BATCH_SIZE)

    train_loss, train_acc = train_running_loss/len(train_iterator), train_running_acc/len(train_iterator)
    epoch_loss, epoch_acc = evaluate(model, valid_iterator, criterion)

    print(f'Epoch {epoch}:')
    print(f'Train Loss: {train_loss:.4f} | Train Accuracy: {train_acc:.2f}% | Epoch Loss: {epoch_loss:.4f} | Epoch Accuracy: {epoch_acc:.2f}%\n')

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/274 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Epoch 0:
Train Loss: 0.7006 | Train Accuracy: 49.75% | Epoch Loss: 0.7107 | Epoch Accuracy: 50.78%



  0%|          | 0/274 [00:00<?, ?it/s]

...

Epoch 22:
Train Loss: 0.6994 | Train Accuracy: 49.85% | Epoch Loss: 0.7023 | Epoch Accuracy: 49.66%



  0%|          | 0/274 [00:00<?, ?it/s]

Epoch 23:
Train Loss: 0.6978 | Train Accuracy: 50.12% | Epoch Loss: 0.7104 | Epoch Accuracy: 50.78%



  0%|          | 0/274 [00:00<?, ?it/s]

Epoch 24:
Train Loss: 0.6980 | Train Accuracy: 49.53% | Epoch Loss: 0.7004 | Epoch Accuracy: 49.30%



In [ ]:
test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.2f}%')

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Test Loss: 0.7119 | Test Acc: 46.03%


Какой-то не очень реультат, но перепробовав кучу разных параметров и потратив несколько часов на проверку я ничего лучше не получил, поэтому буду довольствоваться синицей в руках.